In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
from google.colab import files
uploaded = files.upload()

KeyboardInterrupt: 

In [ ]:
mp = "/content/МП _ Достижение _ Февраль 25.xlsx"
df_mp = pd.read_excel(mp, sheet_name=0, header=4)
metki = "/content/Достижение-Метки_UTM (1).xlsx"
df_metki = pd.read_excel(metki, sheet_name=0, header=5)

/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
df = df_mp[['№', 'Название сайта', 'Период', 'Общая стоимость с учетом НДС и АК', 'KPI прогноз']].copy()
df = df.replace('-', '0')

In [ ]:
df['Категория'] = df['Название сайта'].where(df['№'].isna())
df['Категория'] = df['Категория'].ffill()
df = df[~(df['Период'].isna())]
print(df)

     № Название сайта                   Период  \
1  1.0         Realty    01.02.2025-14.02.2025   
2  2.0        2Realty    03.02.2025-09.02.2025   
4  4.0      OneTarget  01.02.2025 - 28.02.2025   

   Общая стоимость с учетом НДС и АК KPI прогноз               Категория  
1                           342720.0           3  Тематические площадки   
2                              100.0           1  Тематические площадки   
4                           396180.0           0     Охватное размещение  


In [ ]:
# Функция для извлечения начальной и конечной даты
def extract_dates(period):
    start_date, end_date = period.split('-')
    start_date = pd.to_datetime(start_date.strip(), format='%d.%m.%Y')
    end_date = pd.to_datetime(end_date.strip(), format='%d.%m.%Y')
    return start_date, end_date

# Применение функции и создание новых столбцов с начальной и конечной датой
df[['Start Date', 'End Date']] = df['Период'].apply(extract_dates).apply(pd.Series)

# Функция для корректного распределения бюджета по неделям
def calculate_budget_per_week(row):
    start_date = row['Start Date']
    end_date = row['End Date']

    # Начинаем с первой недели, которая содержит start_date
    week_start = start_date - pd.Timedelta(days=start_date.weekday())  # Понедельник недели, в которой start_date
    weeks = []

    while week_start <= end_date:
        # Определяем конец недели (воскресенье)
        week_end = week_start + pd.Timedelta(days=6)

        # Если начало недели в прошлом месяце, но есть дни в текущем, берём только текущий месяц
        if week_start.month < start_date.month:
            week_start = start_date  # Сдвигаем начало недели на первый день периода

        # Если конец недели выходит за границы периода, ограничиваем его
        if week_end > end_date:
            week_end = end_date

        # Количество дней, попадающих в период
        days_in_week = (week_end - week_start).days + 1

        # Общие дни в периоде
        total_days = (end_date - start_date).days + 1

        # Пропорциональный бюджет
        week_budget = row['Общая стоимость с учетом НДС и АК'] * (days_in_week / total_days)

        # Добавляем данные
        weeks.append((week_start, week_end, week_budget))

        # Следующая неделя
        week_start = week_end + pd.Timedelta(days=1)

    return weeks

# Применение функции для всех строк
week_budget_data = []
for idx, row in df.iterrows():
    week_budget_data.extend(calculate_budget_per_week(row))

# Создаём DataFrame для распределённых бюджетов по неделям
df_week_budget = pd.DataFrame(week_budget_data, columns=['Неделя с', 'Неделя по', 'Бюджет на неделю'])

# Добавляем информацию о сайте и периоде для каждой недели
df_week_budget['№'] = np.repeat(df['№'].values, [len(calculate_budget_per_week(row)) for _, row in df.iterrows()])
df_week_budget['Название сайта'] = np.repeat(df['Название сайта'].values, [len(calculate_budget_per_week(row)) for _, row in df.iterrows()])
df_week_budget['Категория'] = np.repeat(df['Категория'].values, [len(calculate_budget_per_week(row)) for _, row in df.iterrows()])
df_week_budget['KPI прогноз'] = np.repeat(df['KPI прогноз'].values, [len(calculate_budget_per_week(row)) for _, row in df.iterrows()])
df_week_budget['Период'] = np.repeat(df['Период'].values, [len(calculate_budget_per_week(row)) for _, row in df.iterrows()])

# Группировка по категории и неделе, суммирование бюджета
df_weekly_category_budget = df_week_budget.groupby(['Категория', 'Неделя с', 'Неделя по'], as_index=False)['Бюджет на неделю'].sum()

# Вывод результата
print(df_weekly_category_budget)

                Категория   Неделя с  Неделя по  Бюджет на неделю
0     Охватное размещение 2025-02-01 2025-02-02      28298.571429
1     Охватное размещение 2025-02-03 2025-02-09      99045.000000
2     Охватное размещение 2025-02-10 2025-02-16      99045.000000
3     Охватное размещение 2025-02-17 2025-02-23      99045.000000
4     Охватное размещение 2025-02-24 2025-02-28      70746.428571
5  Тематические площадки  2025-02-01 2025-02-02      48960.000000
6  Тематические площадки  2025-02-03 2025-02-09     171460.000000
7  Тематические площадки  2025-02-10 2025-02-14     122400.000000


In [ ]:
# --- Чтение заголовка с периодом из первой строки Excel ---
# Считываем файл без заголовков (чтобы получить всю первую строку)
df_raw = pd.read_excel("/content/Достижение-Метки_UTM (1).xlsx", header=None)
header_str = str(df_raw.iloc[0, 0])  # предполагается, что заголовок находится в ячейке A1

# Извлекаем даты отчётного периода с помощью регулярного выражения
match = re.search(r'Отчет за период с (\d{4}-\d{2}-\d{2}) по (\d{4}-\d{2}-\d{2})', header_str)
if match:
    report_start = pd.to_datetime(match.group(1))
    report_end = pd.to_datetime(match.group(2))
else:
    raise ValueError("Не удалось извлечь период из заголовка!")

print(f"Отчетный период: с {report_start.date()} по {report_end.date()}")


Отчетный период: с 2025-02-24 по 2025-03-02


/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
# Оставляем только строки, где 'UTM Campaign' содержит 'arwm'
df_filtered = df_metki[df_metki['UTM Campaign'].astype(str).str.contains('arwm', na=False, case=False)]
# Удаляем строки, где 'UTM Source' содержит 'yandex_maps' или 'navigator'
df_filtered = df_filtered[~df_filtered['UTM Source'].astype(str).isin(['yandex_maps', 'navigator'])]

print(df_filtered)

       UTM Source           UTM Campaign  Визиты  Посетители    Отказы  \
10  rtb_onetarget  arwm|dst|february2025     342         334  0.362573   
16  rtb_onetarget     arwm|dst|march2025     108          94  0.157407   

    Глубина просмотра Время на сайте  Роботность  \
10           2.040936       00:01:58    0.073099   
16           1.814815       00:01:55    0.268519   

    Достижения цели (Кнопка // Скачать презентацию // Клик)  \
10                                                  0         
16                                                  0         

    Достижения цели (Общая цель // Формы)  Достижения цели (Уникальный звонок)  
10                                      0                                    1  
16                                      0                                    0  


In [ ]:

# Преобразуем столбец "время на сайте" (например, "0:01:15") в тип timedelta
df_filtered['Время на сайте'] = pd.to_timedelta(df_filtered['Время на сайте'])

# Суммарное количество визитов и посетителей
total_visits = df_filtered['Визиты'].sum()
total_visitors = df_filtered['Посетители'].sum()

# Считаем взвешенные средние. В качестве веса используем значение в столбце "визиты".
weighted_avg_otkazy = (df_filtered['Отказы'] * df_filtered['Визиты']).sum() / total_visits
weighted_avg_glubina = (df_filtered['Глубина просмотра'] * df_filtered['Визиты']).sum() / total_visits
weighted_avg_robotnost = (df_filtered['Роботность'] * df_filtered['Визиты']).sum() / total_visits

# Для "время на сайте": вычисляем общее время в секундах с учетом веса визитов
weighted_avg_time_sec = (df_filtered['Время на сайте'].dt.total_seconds() * df_filtered['Визиты']).sum() / total_visits

# Функция для форматирования секунд в строку вида H:MM:SS
def format_seconds(total_seconds):
    total_seconds = int(total_seconds)
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    seconds = total_seconds % 60
    return f"{hours}:{minutes:02d}:{seconds:02d}"

weighted_avg_time_str = format_seconds(weighted_avg_time_sec)

# Форматирование показателей "отказы" и "роботность" как проценты (с двумя знаками после запятой)
weighted_avg_otkazy = f"{weighted_avg_otkazy * 100:.2f}%"
weighted_avg_robotnost = f"{weighted_avg_robotnost *100:.2f}%"

# Собираем результаты в итоговый DataFrame
df_summary = pd.DataFrame({
    'визиты': [total_visits],
    'посетители': [total_visitors],
    'отказы': [weighted_avg_otkazy],
    'глубина просмотра': [weighted_avg_glubina],
    'роботность': [weighted_avg_robotnost],
    'время на сайте': [weighted_avg_time_str]
})

print(df_summary)

   визиты  посетители  отказы  глубина просмотра роботность время на сайте
0     450         428  31.33%           1.986667     12.00%        0:01:57


In [ ]:
# Фильтруем строки, где хотя бы одна дата совпадает
report_week_df = df_week_budget[
    (df_week_budget['Неделя с'] == report_start) |
    (df_week_budget['Неделя по'] == report_end) |
    ((df_week_budget['Неделя с'] <= report_start) & (df_week_budget['Неделя по'] >= report_end))
]

# Проверяем, есть ли найденные строки
if report_week_df.empty:
    raise ValueError("Не найдена строка с нужной неделей в бюджете!")

# Извлекаем бюджет для "Тематических площадок" и "Охватного размещения"
tp_budget = report_week_df.loc[report_week_df['Категория'] == 'Тематические площадки', 'Бюджет на неделю']
oh_budget = report_week_df.loc[report_week_df['Категория'] == 'Охватное размещение', 'Бюджет на неделю']

# Преобразуем в число (если строка есть, иначе 0)
tp_budget = tp_budget.values[0] if not tp_budget.empty else 0
oh_budget = oh_budget.values[0] if not oh_budget.empty else 0

# Форматируем суммы в читаемый вид
tp_budget_str = f"{tp_budget:,.2f}".replace(',', ' ') if tp_budget > 0 else "0"
oh_budget_str = f"{oh_budget:,.2f}".replace(',', ' ') if oh_budget > 0 else "0"

# Форматируем даты в нужный вид (дд.мм-дд.мм)
report_period_str = f"{report_start.strftime('%d.%m')}-{report_end.strftime('%d.%m')}"

# Подстановка в отчёт
report_text = f"""
**Медийная реклама {report_period_str}**

**Тематические площадки:**
Выполнение по бюджету плановое ({tp_budget_str} ₽ с НДС)
Первичные обращения — { 'нет данных' }
Целевые обращения — { 'нет данных' }
CPL (первичных обращений) — { 'нет данных' } ₽ с НДС

**Охват:**
Выполнение по бюджету плановое ({oh_budget_str} ₽ с НДС)
Первичные обращения — { 'нет данных' }
Целевые обращения — { 'нет данных' }
CPL (первичных обращений) — { 'нет данных' } ₽ с НДС

**Комментарий:**
Расход на дату — 100 %
Отказы — {weighted_avg_otkazy_pct}
Глубина просмотра — {weighted_avg_glubina:.2f}
Время на сайте — {weighted_avg_time_str}
Роботность — {weighted_avg_robotnost_pct}

**Проделанные работы:**
- Запуск РК
- Написали площадкам по оптимизации РК

**Плановые работы:**
- Следить за динамикой открута и выполнением по ЦО
"""

print(report_text)


**Медийная реклама 24.02-02.03**

**Тематические площадки:**
Выполнение по бюджету плановое (0 ₽ с НДС)
Первичные обращения — нет данных
Целевые обращения — нет данных
CPL (первичных обращений) — нет данных ₽ с НДС

**Охват:**
Выполнение по бюджету плановое (70 746.43 ₽ с НДС)
Первичные обращения — нет данных
Целевые обращения — нет данных
CPL (первичных обращений) — нет данных ₽ с НДС

**Комментарий:**
Расход на дату — 100 %
Отказы — 31.33%
Глубина просмотра — 1.99
Время на сайте — 0:01:57
Роботность — 12.00%

**Проделанные работы:**
- Запуск РК
- Написали площадкам по оптимизации РК

**Плановые работы:**
- Следить за динамикой открута и выполнением по ЦО

